### Model Parameter 설정

In [2]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

### 1) 모델 클래스 유형

#### LLM

In [4]:
from langchain_openai import OpenAI

# model을 지정하지 않아도 실행되는 이유는 LangChain이 내부적으로 기본값을 설정함
llm = OpenAI()
print(llm.model_name)  # 기본 모델 확인

result = llm.invoke("한국의 대표적인 관광지 3군데를 추천해 주세요.")
print(type(result))
print(result)

gpt-3.5-turbo-instruct
<class 'str'>


1. 경복궁
경복궁은 조선 왕조의 주요 궁궐로, 서울에 위치하고 있습니다. 국내외 관광객들에게 인기 있는 곳으로, 조선 왕실의 역사와 문화를 살펴볼 수 있습니다. 특히 봄과 가을에는 아름다운 조경과 함께 청와대 앞 국민청원 체험, 전통문화 체험 등 다양한 이벤트가 열립니다.

2. 제주도
제주도는 우리나라의 남쪽에 위치한 섬으로, 자연이 아름다운 곳으로 유명합니다. 섬 전체가 유네스코 세계자연유산으로 지정되어 있으며, 화산섬이기 때문에 다양한 화산


#### ChatModel

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("human", "{user_input}"),
])

chain = chat_prompt | chat
result = chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해 주세요."})

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
안녕하세요! 한국의 대표적인 관광지로는 다음 세 군데를 추천해 드립니다.

1. 경복궁: 서울에 위치한 경복궁은 조선 시대 왕궁 중 하나로, 아름다운 전통 건축물과 정원이 있는 곳입니다. 궁궐 내부를 구경하며 한국의 역사와 문화를 느껴보세요.

2. 부산 해운대 해수욕장: 부산에 위치한 해운대 해수욕장은 한국에서 가장 유명한 해변 중 하나로, 아름다운 바다 풍경과 다양한 레저 활동이 즐길 수 있는 곳입니다.

3. 제주도: 한국의 대표적인 여행지인 제주도는 풍부한 자연 경관과 다채로운 관광 명소로 유명합니다. 한라산 등반, 성산일출봉, 용두암 등을 방문하여 제주도의 아름다움을 만끽해보세요.

이 세 군데를 방문하면 한국의 다채로운 매력을 느끼실 수 있을 것입니다. 즐거운 여행 되세요!


### 2) 모델 파라미터 설정
##### Temperature 효과
* 0.2 (낮은 값): 더 예측 가능하고 일반적인 이야기
* 1.2 (높은 값): 새로운 요소가 등장하며, 더 창의적이고 흥미로운 이야기 생성

##### Presence Penalty 효과
* 0.0 (낮은 값): 기존에 자주 등장하는 단어와 구조 유지
* 1.5 (높은 값): 새로운 표현과 독창적인 아이디어 등장

In [7]:
from langchain_openai import ChatOpenAI

#  보수적인 설정 (일관된, 논리적인 이야기)
llm_before = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=0.2,  # 낮은 온도로 예측 가능한 출력
    presence_penalty=0.0,  # 기존 패턴 유지
    frequency_penalty=0.0,  # 반복 허용
    max_tokens=150,  # 출력 길이 제한
    top_p=1.0  # 확률 상위 100% 내에서 선택 (제한 없음)
)

#  창의적인 설정 (더 독창적이고 예측 불가능한 이야기)
llm_after = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    temperature=1.2,  # 높은 온도로 창의적인 답변 유도
    presence_penalty=1.2,  # 새로운 단어와 개념 유도
    frequency_penalty=0.5,  # 반복을 억제하여 더 다양한 표현 생성
    max_tokens=300,  # 더 긴 이야기 허용
    top_p=0.8  # 제한 없이 다양한 단어 선택 가능
)

# 질문 설정: 짧은 판타지 이야기 생성
# question = "마법의 세계에서 용이 인간과 친구가 되는 짧은 이야기를 써 주세요."
question = "마법의 세계에서 벌어지는 예상치 못한 사건을 주제로 독창적인 짧은 이야기를 만들어 주세요."

# 모델 호출
response_before = llm_before.invoke(question)
response_after = llm_after.invoke(question)

# 결과 출력
print(" Before (논리적이고 보수적인 이야기)")
print(response_before.content)

print("\n-------------------------\n")

print(" After (창의적인 이야기, 더 풍부한 표현)")
print(response_after.content)

 Before (논리적이고 보수적인 이야기)
한 때 마법사로 유명했던 마을의 주인공인 에밀리아는 이제는 나이 많아져 힘이 약해진 상태였다. 그러나 그녀는 여전히 마을 사람들에게 존경받는 존재였다.

어느 날, 마을에 이상한 소문이 퍼지기 시작했다. 누군가가 마을 주변을 둘러싼 숲에 마법을 부리고 있다는 것이었다. 에밀리아

-------------------------

 After (창의적인 이야기, 더 풍부한 표현)
한 때 마법학교에서 유명했던 학생이었던 리사는 어느 날 자신의 마법 능력을 잃게 되었다. 그녀는 점차 절망에 빠져갔고 결국 마법 세계를 떠나기로 결심했다.

그러나 어느 날, 그녀가 살던 작은 마을에 예상치 못한 일이 벌어졌다. 마을 주변에 있는 숲이 갑자기 사라지고 많은 주민들이 실종되었다. 모두 이것이 악당의 소행이라고 생각했지만, 리사는 다른 이유가 있을 것이라고 직감했다.

리사는 자신의 과거를 돌아보며 마음을 다잡았다. 그리고 그녀는 다시 한번 마법을 사용해 보기로 했다. 그녀는 자신의 능력을 되찾기 위해 열심히 연습하


#### Model의 종류에 따라 결과 값이 다름
* gpt-4o vs gpt-3.5-turbo-0125

In [ ]:
from langchain_openai import ChatOpenAI

# 변경된 설정 (더 창의적인 답변, 새로운 아이디어 유도)
#llm_after = ChatOpenAI(model="gpt-4o", temperature=1.0, presence_penalty=1.5)
llm_after = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.0, presence_penalty=1.5)

# 질문 설정
question = "한국에서 가볼 만한 여행지를 추천해 주세요."

# 모델 호출
response_after = llm_after.invoke(question)

# 결과 출력
print("\n-------------------------\n")
print(" After (창의적인 응답, 새로운 아이디어 포함)")
print(response_after.content)


-------------------------

 After (창의적인 응답, 새로운 아이디어 포함)
한국에는 다양한 매력을 가진 여행지가 많습니다. 다음은 한국에서 가볼 만한 추천 여행지입니다:

1. **서울**
   - 경복궁: 조선시대의 대표적인 궁궐로, 한국 전통 문화를 느낄 수 있습니다.
   - 북촌 한옥마을: 아름다운 한옥들이 모여 있는 지역으로, 전통과 현대가 어우러진 매력을 경험할 수 있습니다.
   - 남산 타워: 서울의 아름다운 야경을 감상할 수 있는 명소입니다.

2. **부산**
   - 해운대 해수욕장: 부산을 대표하는 해변으로, 여름철 많은 관광객이 찾는 곳입니다.
   - 감천문화마을: 형형색색의 집들이 계단식으로 배치된 독특한 마을입니다.
   - 자갈치시장: 신선한 해산물을 맛볼 수 있는 전통 시장입니다.

3. **경주**
   - 불국사: 유네스코 세계문화유산으로 지정된 사찰로, 섬세한 건축미를 자랑합니다.
   - 석굴암: 불국사와 함께 방문할 수 있는 유네스코 문화유산으로, 정교한 불상이 있는 석굴입니다.
   - 안압지: 밤에 조명이 비추면 더욱 아름다워지는 연못입니다.

4. **전주**
   - 전주 한옥마을: 잘 보존된 한옥들 사이에서 전통 문화를 체험할 수 있는 장소입니다.
   - 전주비빔밥: 전주 여행 중 꼭 맛봐야 할 음식입니다.

5. **제주도**
   - 한라산: 등산하며 제주도의 자연을 만끽할 수 있습니다.
   - 성산일출봉: 일출 명소로 유명하며, 정상에서 보는 풍경이 일품입니다.
   - 우도: 아름다운 바다와 독특한 자연경관을 즐길 수 있는 작은 섬입니다.

6. **강릉**
   - 경포대: 동해안 최고의 해변 중 하나로, 여름철 피서지로 인기입니다.
   - 안목해변: 커피 거리가 유명하며, 해변에서 여유로운 시간을 보낼 수 있습니다.

각 지역마다 특별한 볼거리와 즐길거리가 있으니, 취향에 맞춰 방문해 보세요!


##### 2-1) 모델에 직접 파라미터를 전달 (모델 생성 시점)
##### Before / After 파라미터 차이
* temperature: Before(0.7) → 다양성 높은 추천 / After(0.3) → 정확한 일정 제공
* max_tokens: Before(300) → 간략한 응답 / After(800) → 세부 일정 포함
* frequency_penalty: Before(0.5) → 단어 반복 억제 / After(0.2) → 좀 더 자연스러운 응답
* presence_penalty: Before(0.5) → 다양한 장소 추천 / After(0.3) → 새로운 정보 포함

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
question_before = "가족과 함께 한국에서 여행하기 좋은 곳 3곳을 추천해 주세요."
question_after = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": question_before})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": question_after})

#  결과 출력
print("🔹 [Before] 기본적인 여행지 추천")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print("🔹 [After] 맞춤형 일정 및 상세 추천")
print(after_response.content)

🔹 [Before] 기본적인 여행지 추천
가족과 함께 여행하기 좋은 한국의 여행지로는 다음 세 곳을 추천해 드립니다:

1. 제주도: 자연 풍경이 아름다운 제주도는 가족 여행객들에게 인기 있는 여행지입니다. 해변, 화산 동굴, 폭포 등 다양한 관광 명소와 맛집이 있어 가족 모두가 즐길 수 있습니다.

2. 서울: 현대적인 도시 생활과 전통 문화를 동시에 경험할 수 있는 서울은 다양한 관광 명소와 먹거리, 쇼핑 장소가 모두 갖춰져 있어 가족 단위로 다양한 활동을 즐길 수 있습니다.

3. 경주: 역사와 문화가 곳곳에 묻어나는 경주는 한국의 역사적인 유적지와 문화유산을 만나볼 수 있는 곳으로, 가


🔹 [After] 맞춤형 일정 및 상세 추천
가족과 함께 여유롭게 즐길 수 있는 3박 4일 일정을 제안해 드리겠습니다. 한국의 대표적인 관광지를 방문하면서 가족 모두가 즐거운 추억을 만들 수 있는 일정을 준비했습니다.

**1일차: 서울**
- 아침: 인천 공항 도착 후 서울로 이동
- 오전: 경복궁, 광화문 광장, 청계천 산책
- 점심: 인사동에서 전통 음식 즐기기
- 오후: 북촌 한옥마을, 이화벽화마을 방문
- 저녁: 명동에서 쇼핑 및 저녁 식사

**2일차: 강원도(강릉)**
- 아침: 서울에서 강릉으로 이동
- 오전: 오죽헌, 경포해변 관광
- 점심: 강릉시내 맛집에서 강릉오징어회 등 현지 음식 즐기기
- 오후: 속초로 이동하여 속초 해수욕장 즐기기
- 저녁: 강릉 현지 맛집에서 저녁 식사

**3일차: 강원도(평창)**
- 아침: 강릉에서 평창으로 이동
- 오전: 알펜시아 리조트 스키 체험 또는 알펜시아 스노우파크 방문
- 점심: 평창 현지 맛집에서 식사
- 오후: 평창 올림픽파크 관광
- 저녁: 평창 시내에서 저녁 식사

**4일차: 서울**
- 아침: 평창에서 서울로 이동
- 오전: 남산서울타워 전망대에서 서울 전경 감상
- 점심: 남대문 시장 현지 음식 즐기기
- 오후: 명동이나 동대문 쇼핑 또는 한복체험
- 오후 5시 이후에는 인천공항으로 이동하여 귀

#### 2-2) 모델에 추가적인 파라미터를 전달
- bind() 메서드를 사용
- max_tokens=50 (기본) / max_tokens=150 (상세한 설명) 
- stop=["."] → 첫 번째 마침표에서 응답을 중단
- temperature=0.8 → 보다 창의적인 답변을 생성함

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한

#  질문 설정
question = "태양계에서 가장 큰 행성은 무엇이며, 그 특징을 설명해 주세요."

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n🔹 [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print("🔹 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print("🔹 [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)


🔹 [Before] 기본 max_tokens=150 (기본 답변)
태양계에서 가장 큰 행성은 목성입니다. 목성은 가스 행성으로, 대기 중에 수소와 헬륨이 주로 구성되어 있습니다. 목성은 태양계에서 가장 많은 자기를 가지고 있는 행성이며, 그것은 목성의 자기장이 강하게 발달해 있기 때문입니다. 또한 목성은 태양계에서 가장 많은 위성을 가지고 있습니다. 목성의 가스 행성은 대기가


🔹 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)
태양계에서 가장 큰 행성은 목성입니다


🔹 [After] max_tokens=150, temperature=0.8 (창의적인 답변)
태양계에서 가장 큰 행성은 목성입니다. 목성은 태양 주위를 공전하는 행성 중 가장 크고 질량이 가장 많은 행성으로 알려져 있습니다. 목성은 지름이 약 142,984km로 지구의 약 11배에 달하며, 질량은 약 1,898 x 10^24kg로 지구의 약 318배에 달합니다.

목성의 대기는 수소와 헬륨으로 이루어져 있으
